In [1]:
import os
import re



In [2]:
RUST_SRC_DIR = "c:/users/keivf/.rustup/toolchains/stable-x86_64-pc-windows-msvc/lib/rustlib/src/rust/library"  # adjust this if you cloned rust-lang/rust elsewhere
TARGET_STRUCT = "Vec"




In [3]:
def find_impl_blocks(filepath, struct_name):
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    impl_blocks = []
    inside_impl = False
    brace_count = 0
    current_block = []

    for i, line in enumerate(lines):
        if not inside_impl:
            if re.match(rf'\s*impl(\s*<[^>]*>)?\s*{re.escape(struct_name)}', line):
                inside_impl = True
                brace_count = line.count('{') - line.count('}')
                current_block = [line]
        else:
            brace_count += line.count('{') - line.count('}')
            current_block.append(line)
            if brace_count <= 0:
                impl_blocks.append("".join(current_block))
                inside_impl = False

    return impl_blocks

def extract_fn_names(impl_block):
    fn_pattern = re.compile(r'\bfn\s+(\w+)\s*[\(<]')
    return fn_pattern.findall(impl_block)

def find_vec_functions():
    vec_functions = []
    for root, _, files in os.walk(os.path.join(RUST_SRC_DIR, "alloc", "src", "vec")):
        for file in files:
            if file.endswith(".rs"):
                path = os.path.join(root, file)
                impl_blocks = find_impl_blocks(path, TARGET_STRUCT)
                for block in impl_blocks:
                    functions = extract_fn_names(block)
                    if functions:
                        vec_functions.extend(functions)
    return sorted(set(vec_functions))
    

In [4]:
functions = find_vec_functions()
print(f"Found {len(functions)} functions in impl Vec:")
for f in functions:
    print(f"  - {f}")

Found 71 functions in impl Vec:
  - allocator
  - append
  - append_elements
  - as_mut_ptr
  - as_mut_slice
  - as_non_null
  - as_ptr
  - as_slice
  - assert_failed
  - capacity
  - clear
  - dedup
  - dedup_by
  - dedup_by_key
  - deflateGetDictionary
  - drain
  - drop
  - extend_desugared
  - extend_from_slice
  - extend_from_within
  - extend_trusted
  - extend_with
  - extract_if
  - from_iter_fallible
  - from_parts
  - from_parts_in
  - from_raw_parts
  - from_raw_parts_in
  - get_dictionary
  - insert
  - into_boxed_slice
  - into_flattened
  - into_parts
  - into_parts_with_alloc
  - into_raw_parts
  - into_raw_parts_with_alloc
  - is_empty
  - leak
  - len
  - main
  - new
  - new_in
  - pop
  - pop_if
  - process_data
  - process_loop
  - push
  - push_within_capacity
  - remove
  - reserve
  - reserve_exact
  - resize
  - resize_with
  - retain
  - retain_mut
  - set_len
  - shrink_to
  - shrink_to_fit
  - spare_capacity_mut
  - splice
  - split_at_spare_mut
  - split_at_